In [1]:
# Suppress all HuggingFace/transformers warnings
import warnings
warnings.filterwarnings("ignore")            

from transformers import logging as hf_logging
hf_logging.set_verbosity_error()             # only show errors, not warnings

In [2]:
# ─── 0) Monkey‑patch for RAGAS exceptions ───────────────────────────────────────
import ragas.exceptions
if not hasattr(ragas.exceptions, "MaxRetriesExceeded"):
    class MaxRetriesExceeded(Exception):
        pass
    ragas.exceptions.MaxRetriesExceeded = MaxRetriesExceeded

In [3]:
# ─── 1) Standard imports ───────────────────────────────────────────────────────
import warnings, time, re
warnings.filterwarnings("ignore", message="Token indices sequence length")
import pandas as pd
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import context_precision, context_recall, faithfulness, answer_similarity


In [4]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from langchain.llms.base import LLM

# 1) Load & configure tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.model_max_length = 1024
tokenizer.truncation_side   = "right"
tokenizer.pad_token         = tokenizer.eos_token

# 2) Load model on CPU (or GPU if available)
device = "cuda" if torch.cuda.is_available() else "cpu"
model  = GPT2LMHeadModel.from_pretrained("gpt2").to(device)

# 3) The updated wrapper
class LocalGPT2(LLM):
    """GPT‑2 wrapper that truncates at 1024 tokens and generates 50 new tokens."""
    def _call(self, prompt: str, stop=None, run_manager=None, **kwargs) -> str:
        # Tokenize + truncate
        inputs = tokenizer(
            prompt,
            return_tensors="pt",
            truncation=True,
            max_length=tokenizer.model_max_length,
        ).to(device)
        # Generate up to 50 new tokens
        outputs = model.generate(**inputs, max_new_tokens=50)
        # Decode and return text
        return tokenizer.decode(outputs[0], skip_special_tokens=True)

    @property
    def _identifying_params(self) -> dict:
        return {"model_name": "gpt2-trunc50"}

    @property
    def _llm_type(self) -> str:
        return "local_gpt2"

# 4) Instantiate
llm = LocalGPT2()

In [5]:
# ─── 3) Embeddings setup ───────────────────────────────────────────────────────
from langchain_community.embeddings import HuggingFaceEmbeddings
embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={"device": "cpu"},
)

/var/folders/c4/jsnm59wd4hsgp59byfhnrs3w0000gn/T/ipykernel_77344/3965307048.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(


In [6]:
# ─── 4) Load test‐set & prepare Dataset ────────────────────────────────────────
test_df = pd.read_csv("/Users/basusmac/Desktop/Github Repositories/SmartCandidate-Analyzer-RAG-Based-Resume-Screening/data/main-data/gpt4-ragfusion/test-results/testres-5.csv")
# split your stored contexts back into a list
test_df["contexts"] = test_df["contexts"].apply(lambda c: c.split("==="))
test_dataset = Dataset.from_pandas(test_df)

In [8]:
# ─── 5) Run RAGAS evaluation loop ───────────────────────────────────────────────
result_list = []
for i in range(len(test_dataset)):
    single = test_dataset.select([i])
    metrics_df = evaluate(
        single,
        llm=llm,
        embeddings=embedding_model,
        metrics=[context_precision, context_recall, faithfulness, answer_similarity],
    ).to_pandas()
    result_list.append(metrics_df)
    if i == 9:
        time.sleep(60)

Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

Exception raised in Job[1]: IndexError(index out of range in self)
Exception raised in Job[0]: IndexError(index out of range in self)
Exception raised in Job[2]: IndexError(index out of range in self)


Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

Exception raised in Job[0]: IndexError(index out of range in self)
Exception raised in Job[1]: IndexError(index out of range in self)
Exception raised in Job[2]: TimeoutError()


Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

...

Exception raised in Job[1]: IndexError(index out of range in self)
Exception raised in Job[2]: IndexError(index out of range in self)
Exception raised in Job[0]: IndexError(index out of range in self)


Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

Exception raised in Job[2]: IndexError(index out of range in self)
Exception raised in Job[0]: IndexError(index out of range in self)
Exception raised in Job[1]: IndexError(index out of range in self)


Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

Exception raised in Job[2]: IndexError(index out of range in self)
Exception raised in Job[0]: IndexError(index out of range in self)
Exception raised in Job[1]: TimeoutError()


In [9]:
# ─── 6) Combine & save ─────────────────────────────────────────────────────────
final_df = pd.concat(result_list, ignore_index=True)
print(final_df.head())
final_df.to_csv(
    "/Users/basusmac/Desktop/Github Repositories/SmartCandidate-Analyzer-RAG-Based-Resume-Screening/data/main-data/gpt4-ragfusion/evaluation-results/evaluation-result-5.csv",
    index=False
)

                                          user_input  \
0  Node js developer\nExperience\n2-3 years\nSkil...   
1  Backend Developer\nJob description\nUse markup...   
2  Network Administrator\nJob DescriptionConfigur...   
3  Backend Developer\nJOB DESCRIPTION: -\nProject...   
4  Java Developer\nInfosys is seeking Backend Dev...   

                                  retrieved_contexts  \
0  [Applicant ID 176\n: \n\nObjective: Seeking a ...   
1  [Applicant ID 264\n: \n\nName: John Doe\nEmail...   
2  [Applicant ID 97\n: \n\nName: John Doe\n\nCont...   
3  [Applicant ID 23\n - \n\nSUMMARY\nA Backend De...   
4  [Applicant ID 104\n ():\nName: Jane Smith\n\nC...   

                                            response  \
0  Based on the job description provided, the bes...   
1  The best candidate for this Backend Developer ...   
2  The best candidate for the Network Administrat...   
3  The best candidate for the Backend Developer p...   
4  Based on the job description provided by In